# Azure ML - Registro de modelos con mlflow

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID, workspace_name)

## Registro de modelos con mlflow

Ahora que hemos visto cómo buscar la mejor combinación de hiperparámeetros, vamosa a ver cómo registrar los modelos y así poder usar el mejor modelo en producción

### Regristro de modelos con la interfaz gráfica

Volvemos a dar al botón con el símbolo `+`, le damos a `Create new file`, en `File type` seleccionamos `Notebook (*.ipynb)` y le ponemos el nombre `run_text-clasification-mlfow-model-registry.ipynb`. Ahora copiamos las siguientes celdas de código

```python
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except:
    credential = InteractiveBrowserCredential()
```

```python
ml_client = MLClient.from_config(credential=credential)
```

```python
from azure.ai.ml import command
from azure.ai.ml.sweep import Choice, LogUniform

SUSCRIPTION_ID = ""
RESOURCE_GROUP = ""
WORKSPACE = ""
EXPERIMENT_NAME = "image_classification-mlflow-model-registry"

execute_command = f"python image-classification-mlflow.py --suscription_id {SUSCRIPTION_ID} --resource_group {RESOURCE_GROUP} --workspace_name {WORKSPACE} --experiment_name {EXPERIMENT_NAME} --batch_size ${{{{inputs.batch_size}}}} --epochs ${{{{inputs.epochs}}}} --learning_rate ${{{{inputs.learning_rate}}}}"
inputs = {
    "batch_size": 8,
    "epochs": 1,
    "learning_rate": 1e-2,
}
environment = "cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_GUI@latest"
compute_instance = "compute-instance-GUI"
display_name = "image-classificator"
sampling_algorithm = "random"
primary_metric = "Accuracy"
goal = "Maximize"

# configure base command
job = command(
    code="./",
    command=execute_command,
    inputs=inputs,
    environment=environment,
    compute=compute_instance,
    display_name=display_name,
    experiment_name=EXPERIMENT_NAME
)

# configure sweep
command_job_for_sweep = job(
    batch_size=Choice(values=[8, 16, 32, 64]),
    epochs=Choice(values=range(1,10)),
    learning_rate=LogUniform(-6, -2),
)

# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute=compute_instance,
    sampling_algorithm=sampling_algorithm,
    primary_metric=primary_metric,
    goal=goal,
)

# set the name of the sweep job experiment
sweep_job.experiment_name=EXPERIMENT_NAME

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)
```

Rellena `SUSCRIPTION_ID`, `RESOURCE_GROUP` y `WORKSPACE` con tus valores

```python
returned_job = ml_client.create_or_update(sweep_job)
```

Aquí es dónde registramos el modelo. Cuando haya terminado el entrenamiento ejecutamos el siguiente código

```python
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)
```

```python
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

job_name = returned_job.name
MODEL_NAME = "image-classification-FashionMNIST"
DESCRIPTION = "Model trained to classify FashionMNIST dataset."

run_model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/model/",
    name=MODEL_NAME,
    description=DESCRIPTION,
    type=AssetTypes.MLFLOW_MODEL,
)
# Uncomment after adding required details above
ml_client.models.create_or_update(run_model)
```

### Regristro de modelos con la interfaz gráfica

No he encotrado la manera de ejecutar el script mediante la CLI de Azure ML

### Regristro de modelos con el SDK de Python

Para el registro de modelos ejecutamos el mismo código que ejecutamos en el notebook mediante la interfaz gráfica

In [ ]:
from azure.ai.ml import command
from azure.ai.ml.sweep import Choice, LogUniform

SUSCRIPTION_ID = ""
RESOURCE_GROUP = ""
WORKSPACE = ""
EXPERIMENT_NAME = "image_classification-mlflow-model-registry"

execute_command = f"python image-classification-mlflow.py --suscription_id {SUSCRIPTION_ID} --resource_group {RESOURCE_GROUP} --workspace_name {WORKSPACE} --experiment_name {EXPERIMENT_NAME} --batch_size ${{{{inputs.batch_size}}}} --epochs ${{{{inputs.epochs}}}} --learning_rate ${{{{inputs.learning_rate}}}}"
inputs = {
    "batch_size": 8,
    "epochs": 1,
    "learning_rate": 1e-2,
}
environment = "cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_GUI@latest"
compute_instance = "compute-instance-Python"
display_name = "image-classificator"
sampling_algorithm = "random"
primary_metric = "Accuracy"
goal = "Maximize"

# configure base command
job = command(
    code="./",
    command=execute_command,
    inputs=inputs,
    environment=environment,
    compute=compute_instance,
    display_name=display_name,
    experiment_name=EXPERIMENT_NAME
)

# configure sweep
command_job_for_sweep = job(
    batch_size=Choice(values=[8, 16, 32, 64]),
    epochs=Choice(values=range(1,10)),
    learning_rate=LogUniform(-6, -2),
)

# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute=compute_instance,
    sampling_algorithm=sampling_algorithm,
    primary_metric=primary_metric,
    goal=goal,
)

# set the name of the sweep job experiment
sweep_job.experiment_name=EXPERIMENT_NAME

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Rellena `SUSCRIPTION_ID`, `RESOURCE_GROUP` y `WORKSPACE` con tus valores

In [ ]:
returned_job = ml_client.create_or_update(job)

Aquí es dónde registramos el modelo. Cuando haya terminado el entrenamiento ejecutamos el siguiente código

In [ ]:
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Lo deberíamos poder registrar con el siguiente código, que es el que viene en el ejemplo del propio curso de Azure DP-100

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

job_name = returned_job.name
MODEL_NAME = "image-classification-FashionMNIST"
DESCRIPTION = "Model trained to classify FashionMNIST dataset."

run_model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/model/",
    name=MODEL_NAME,
    description=DESCRIPTION,
    type=AssetTypes.MLFLOW_MODEL,
)
# Uncomment after adding required details above
ml_client.models.create_or_update(run_model)

Pero da error por el `path` y como en la documentación no da mucha información de cómo tiene que ser esa variable. He registrado el modelo con el botón `+ Registry model` de la interfaz gráfica

Si ahora nos vamos a la seción de `Models` de la interfaz gráfica, deberíamos ver el modelo registrado